### Item-base Collaborative Filtering - Retrieval
This notebook demonstrates how to retrieve the recommendations for a user using the Item-based Collaborative Filtering model in production. 

#### Pre-requisites
1. Have the index trained and saved in the `yelp_ItemCF.db` file.

In [1]:
import sqlite3
import pickle

In [2]:
# Connect to the SQLite database
db_path = './yelp_ItemCF.db'
conn = sqlite3.connect(db_path)

In [5]:
# Function to retrieve user-business mappings from the database
def retrieve_user_business_mapping(conn):
    cursor = conn.cursor()

    # Fetch business mappings
    cursor.execute('''SELECT business_id, business_idx FROM business_mapping''')
    business_mapping = {row[0]: row[1] for row in cursor.fetchall()}

    return business_mapping

In [6]:
# Retrieve the user-business mappings
business_mapping = retrieve_user_business_mapping(conn)

In [7]:
# Function to get businesses a user interacted with
def get_user_businesses(user_id, conn):
    cursor = conn.cursor()
    cursor.execute('''SELECT business_id, stars_review FROM user_item_index WHERE user_id = ?''', (user_id,))
    return cursor.fetchall()

In [8]:
# Function to get top-k similar businesses for a given business
def get_top_k_similar_businesses(business_id, k, conn):
    cursor = conn.cursor()
    cursor.execute('''SELECT similarity_vector FROM item_item_similarity WHERE item_id = ?''', (business_id,))
    result = cursor.fetchone()

    if result is None:
        return []

    similarity_vector = pickle.loads(result[0])
    indices, data = similarity_vector

    # Get top-k similar businesses
    top_k = sorted(zip(indices, data), key=lambda x: -x[1])[:k]

    # Map indices to business ids
    similar_businesses = [(list(business_mapping.keys())[idx], score) for idx, score in top_k]

    return similar_businesses

In [9]:
# Function to predict user interests based on similar businesses
def predict_user_interests(user_id, k=10, conn=conn):
    user_businesses = get_user_businesses(user_id, conn)

    recommended_businesses = {}
    for business_id, _ in user_businesses:
        similar_businesses = get_top_k_similar_businesses(business_id, k, conn)

        for similar_business_id, score in similar_businesses:
            if similar_business_id in recommended_businesses:
                recommended_businesses[similar_business_id] += score
            else:
                recommended_businesses[similar_business_id] = score

    # Sort recommendations by score
    recommended_businesses = sorted(recommended_businesses.items(), key=lambda x: -x[1])

    return recommended_businesses[:k]

In [11]:
# get the top 10 users in the database
# user_ids = list(user_mapping.keys())[:10]

user_ids = ['razUB7ciYZluvxWM6shmtw', '3YhG4h4Ok654iVfqdmkuRg', 'VyC2fG4dcMG07nrxh4jLnw', 'Q5jOFJYhIsN8ouJ1rnsLQQ', 'gdcRlubKDmslUYFPHUp1Cg', 'CNeaJDD_ZihiafOvSGSpPw', 'TItDMAEw7_6Nge38jJPspA', 'zxyKewY6p0CjnFprEutaog', 'IsdzdZEH9uHTnwX3acqhtg', 'AC-oO5luq3enSNFDYAVKJA']

for user_id in user_ids:
    recommendations = predict_user_interests(user_id, k=10)
    print(f"Recommendations for user {user_id}: {recommendations}")
    # get the review

# Close the database connection when done
conn.close()

Recommendations for user razUB7ciYZluvxWM6shmtw: []
Recommendations for user 3YhG4h4Ok654iVfqdmkuRg: []
Recommendations for user VyC2fG4dcMG07nrxh4jLnw: []
Recommendations for user Q5jOFJYhIsN8ouJ1rnsLQQ: []
Recommendations for user gdcRlubKDmslUYFPHUp1Cg: []
Recommendations for user CNeaJDD_ZihiafOvSGSpPw: []
Recommendations for user TItDMAEw7_6Nge38jJPspA: []
Recommendations for user zxyKewY6p0CjnFprEutaog: []
Recommendations for user IsdzdZEH9uHTnwX3acqhtg: []
Recommendations for user AC-oO5luq3enSNFDYAVKJA: []
